In [1]:
import pandas as pd
import nltk
import vaderSentiment
import re
import string

from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Read Data from CSV File:

In [2]:
path = r'C:\Users\JZ\Desktop\Data\JPMC_data_csv1.csv'

In [3]:
def load():
    pd.set_option("display.max_colwidth", None) #-1
    return pd.read_csv(path)

In [4]:
def df_style(val):
    return "font-weight: normal"

In [5]:
def df_style2(val2):
    return "font-style: normal"

In [6]:
totalInfo = load().copy()

In [8]:
#fill any NaN/invalid values
totalInfo = totalInfo.fillna(0)

In [9]:
#get rid of any noisy data (compound score is zero)
filter_noise = (totalInfo['compound score '] != '0')
totalInfo = totalInfo.loc[filter_noise]

In [10]:
# Filter special characters
def filter_special_chars(comment):
    new_sentence = '' #empty string
    new_comment = re.sub(r"https:\S+", "", comment) #remove url links from comment

    for i in range(0, len(new_comment)):
        if ord(new_comment[i]) >= 110000: #for characters like 𝐴𝑀𝐶 BB  𝑁𝑂𝐾 
             new_sentence += new_comment[i]
        if ord(new_comment[i]) <= 127: #keep all ascii characters
            new_sentence += new_comment[i]
                
    return new_sentence

Find if sentence has ... in it

In [12]:
def remove_sentence(comment): #if text contains …, then it will be removed (… means truncated)
    obj = re.search('…', comment)
    if obj != None:
        return 1
    else:
        return -1
        

In [13]:
totalInfo['remove'] = totalInfo['text'].map(lambda x: '1' if remove_sentence(x) == 1 else '-1')

Filter out sentences that are truncated:

In [14]:
filter_truncated = totalInfo['remove'] == '-1'
full_comments_df = totalInfo.loc[filter_truncated]
full_comments_df

,Date,text,favorite_count,favorited,retweeted,lang,quoted_status,Stock_Name,sentiment,sentiment_class,compound score,Stock Price,remove
1,10/24/2021,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦",0,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.6825}",positive,0.68,169.8,-1
2,10/24/2021,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,0,FALSE,FALSE,en,0,GME,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}",negative,-0.3,169.8,-1
4,10/24/2021,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,0,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.5423}",positive,0,169.8,-1
5,10/24/2021,Crush them! Good luck to $AMC $GME we will soon unite $DWAC https://t.co/cA8Z1sYXtT,0,FALSE,FALSE,en,"{'created_at': 'Sun Oct 24 17:07:17 +0000 2021', 'id': 1452320805132677125, 'id_str': '1452320805132677125', 'text': 'CITADEL GOES TO COURT TOMORROW TO SAY THEY ARE PROTECTING RETAIL INVESTORS.\n\nKEEP IN MIND THE SEC ALREADY HAS A SYSâ€¦ https://t.co/WQkCUvsLnX', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/WQkCUvsLnX', 'expanded_url': 'https://twitter.com/i/web/status/1452320805132677125', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389833864122613761, 'id_str': '1389833864122613761', 'name': 'AMC 2 Moon', 'screen_name': 'AMC_Apee', 'location': 'MoooooooNðŸš€', 'description': 'FOLLOW ME IF YOU WANNA BE RICH! ðŸ’°ðŸš€', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 18218, 'friends_count': 1932, 'listed_count': 58, 'created_at': 'Wed May 05 06:46:46 +0000 2021', 'favourites_count': 23749, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 12259, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1389833864122613761/1634276127', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': False, 'following': None, 'follow_request_sent': None, 'notifications': None, 'translator_type': 'none', 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 112, 'favorite_count': 500, 'favorited': False, 'retweeted': False, 'lang': 'en'}",GME,"{'neg': 0.09, 'neu': 0.562, 'pos': 0.348, 'compound': 0.68}",positive,0,169.8,-1
7,10/24/2021,@WSBArmy I own $doge $amc $gme $shib $btc $eth and more. Love the power of momentum and community buzz.,0,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.6697}",positive,0.67,169.8,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26666,11/6/2021,RT @TrendS

Filter out special characters for full comments:

In [15]:
full_comments_df['new_text'] = full_comments_df['text'].map(lambda x: filter_special_chars(x)) # remove special char

C:\Users\JZ\AppData\Local\Temp/ipykernel_8044/399114051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_comments_df['new_text'] = full_comments_df['text'].map(lambda x: filter_special_chars(x)) # remove special char


In [16]:
full_comments_df = full_comments_df[['Date','text','new_text','favorited', 'retweeted', 'lang', 'quoted_status', 'Stock_Name', 'sentiment', 'sentiment_class', 'compound score ', 'Stock Price', 'remove']]

In [17]:
full_comments_df

,Date,text,new_text,favorited,retweeted,lang,quoted_status,Stock_Name,sentiment,sentiment_class,compound score,Stock Price,remove
1,10/24/2021,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦","RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!",FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.6825}",positive,0.68,169.8,-1
2,10/24/2021,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,We are going through the beginning stages of everything Ive said in the last year about the economy. Im no genius,FALSE,FALSE,en,0,GME,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}",negative,-0.3,169.8,-1
4,10/24/2021,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,The largest mining company in the world\nGet Bonus 40% \n\n\n$elon $doge $btc $tsla $amc $gme,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.5423}",positive,0,169.8,-1
5,10/24/2021,Crush them! Good luck to $AMC $GME we will soon unite $DWAC https://t.co/cA8Z1sYXtT,Crush them! Good luck to $AMC $GME we will soon unite $DWAC,FALSE,FALSE,en,"{'created_at': 'Sun Oct 24 17:07:17 +0000 2021', 'id': 1452320805132677125, 'id_str': '1452320805132677125', 'text': 'CITADEL GOES TO COURT TOMORROW TO SAY THEY ARE PROTECTING RETAIL INVESTORS.\n\nKEEP IN MIND THE SEC ALREADY HAS A SYSâ€¦ https://t.co/WQkCUvsLnX', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/WQkCUvsLnX', 'expanded_url': 'https://twitter.com/i/web/status/1452320805132677125', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389833864122613761, 'id_str': '1389833864122613761', 'name': 'AMC 2 Moon', 'screen_name': 'AMC_Apee', 'location': 'MoooooooNðŸš€', 'description': 'FOLLOW ME IF YOU WANNA BE RICH! ðŸ’°ðŸš€', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 18218, 'friends_count': 1932, 'listed_count': 58, 'created_at': 'Wed May 05 06:46:46 +0000 2021', 'favourites_count': 23749, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 12259, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1389833864122613761/1634276127', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': False, 'following': None, 'follow_request_sent': None, 'notifications': None, 'translator_type': 'none', 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 112, 'favorite_count': 500, 'favorited': False, 'retweeted': False, 'lang

Go through string and change styling if needed:

In [19]:
full_comments_df

,Date,text,new_text,favorited,retweeted,lang,quoted_status,Stock_Name,sentiment,sentiment_class,compound score,Stock Price,remove
1,10/24/2021,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦","RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!",FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.6825}",positive,0.68,169.8,-1
2,10/24/2021,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,We are going through the beginning stages of everything Ive said in the last year about the economy. Im no genius,FALSE,FALSE,en,0,GME,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}",negative,-0.3,169.8,-1
4,10/24/2021,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,The largest mining company in the world\nGet Bonus 40% \n\n\n$elon $doge $btc $tsla $amc $gme,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.5423}",positive,0,169.8,-1
5,10/24/2021,Crush them! Good luck to $AMC $GME we will soon unite $DWAC https://t.co/cA8Z1sYXtT,Crush them! Good luck to $AMC $GME we will soon unite $DWAC,FALSE,FALSE,en,"{'created_at': 'Sun Oct 24 17:07:17 +0000 2021', 'id': 1452320805132677125, 'id_str': '1452320805132677125', 'text': 'CITADEL GOES TO COURT TOMORROW TO SAY THEY ARE PROTECTING RETAIL INVESTORS.\n\nKEEP IN MIND THE SEC ALREADY HAS A SYSâ€¦ https://t.co/WQkCUvsLnX', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/WQkCUvsLnX', 'expanded_url': 'https://twitter.com/i/web/status/1452320805132677125', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389833864122613761, 'id_str': '1389833864122613761', 'name': 'AMC 2 Moon', 'screen_name': 'AMC_Apee', 'location': 'MoooooooNðŸš€', 'description': 'FOLLOW ME IF YOU WANNA BE RICH! ðŸ’°ðŸš€', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 18218, 'friends_count': 1932, 'listed_count': 58, 'created_at': 'Wed May 05 06:46:46 +0000 2021', 'favourites_count': 23749, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 12259, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1389833864122613761/1634276127', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': False, 'following': None, 'follow_request_sent': None, 'notifications': None, 'translator_type': 'none', 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 112, 'favorite_count': 500, 'favorited': False, 'retweeted': False, 'lang

In [20]:
full_comments_df[['text','new_text']]

,text,new_text
1,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦","RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!"
2,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,We are going through the beginning stages of everything Ive said in the last year about the economy. Im no genius
4,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,The largest mining company in the world\nGet Bonus 40% \n\n\n$elon $doge $btc $tsla $amc $gme
5,Crush them! Good luck to $AMC $GME we will soon unite $DWAC https://t.co/cA8Z1sYXtT,Crush them! Good luck to $AMC $GME we will soon unite $DWAC
7,@WSBArmy I own $doge $amc $gme $shib $btc $eth and more. Love the power of momentum and community buzz.,@WSBArmy I own $doge $amc $gme $shib $btc $eth and more. Love the power of momentum and community buzz.
...,...,...
26666,RT @TrendSpider: $AMZN Rejected at the gap fill on the daily 😬 https://t.co/gfJ1pC0FdJ,RT @TrendSpider: $AMZN Rejected at the gap fill on the daily 😬
26667,RT @TrendSpider: $AMZN Rejected at the gap fill on the daily 😬 https://t.co/gfJ1pC0FdJ,RT @TrendSpider: $AMZN Rejected at the gap fill on the daily 😬
26668,$AMZN Rejected at the gap fill on the daily 😬 https://t.co/gfJ1pC0FdJ,$AMZN Rejected at the gap fill on the daily 😬
26677,Worried about inflation: \n\nThese 6 stocks might fight inflation better as per Morningstar.\n\n$BUD\n$KO\n$STZ\n$MSFT\n$FB\n$AMZN,Worried about inflation: \n\nThese 6 stocks might fight inflation better as per Morningstar.\n\n$BUD\n$KO\n$STZ\n$MSFT\n$FB\n$AMZN


Modifed vaderSentiment Analyzer:

In [21]:
stocks = totalInfo['Stock_Name'].to_numpy()
stocks

all_stocks = set(stocks)
all_stocks.remove('Stock_Name')

all_stocks

{'AMC', 'AMZN', 'BAC', 'FB', 'GME', 'JPM', 'NOK', 'SBUX', 'TSLA'}

In [34]:
def get_sentiment(comment):
    sentence_list = []
    stock_sentiment_dict = {}
    all_stocks_mentioned = defaultdict(list) # ex: GME:[1, 0.68]
                                             # 1 <-- number of times it appeared in comment
                                             # 0.68 <-- sum of compound sentimenent scores
                                             # Later find average of compound scores ex: 0.68/1 = 0.68 (average)
    avg_score = 0
    within_comment = nltk.sent_tokenize(comment) #makes a list of all sentences in comment
    analyzer1 = SentimentIntensityAnalyzer() #Create analyzer object
    full = analyzer1.polarity_scores(within_comment)
    
    for sentence in within_comment:
        stocks_seen = set() #within each sentence, keeps track of what stocks are seen
        analyzer2 = SentimentIntensityAnalyzer() #Create analyzer object
        
        vs = analyzer2.polarity_scores(sentence) # sentiment of sentence
        words = nltk.word_tokenize(sentence) # makes a list of all the words within a sentence
        
        for word in words: #iterates through each word in sentence
            if word in all_stocks: #checks if stock name is a word
                if word not in all_stocks_mentioned: #adds word to dictionary if never seen before
                    if word not in stocks_seen:
                        stock_sentiment_dict[word] = 0
                        
                        stocks_seen.add(word)
                        all_stocks_mentioned[word].append(1)
                        all_stocks_mentioned[word].append(vs['compound'])
                
                else: #if word is already in dictionary (seen before)
                    if word not in stocks_seen:
                        all_stocks_mentioned[word][0] += 1
                        all_stocks_mentioned[word][1] += vs['compound']
            
    #calculate average compound score for each stock
    for name in all_stocks_mentioned.keys():
        stock_sentiment_dict[name] = all_stocks_mentioned[name][1] / all_stocks_mentioned[name][0]
    
    #put average stock sentiment in dictionary
    #stock_sentiment_dict[word] = avg_score
    
    if not stock_sentiment_dict: #if empty
        return -1
    
    #if stocks have 0 as value (means that vadersentiment was used on a sentence that has list of stock names)
    #ex:Don't miss our next FREE OPTION TRADE. Sign up for our Daily Free Trades at  𝑁𝑉𝐷𝐴 TSLA  𝐺𝑆 GILD  𝐷𝐾𝑁𝐺 GOOGL $FB	
    for stock, val in stock_sentiment_dict.items():
        if val == 0: 
            stock_sentiment_dict[stock] = full['compound'] #take sentiment of entire comment as analysis
            
    return stock_sentiment_dict

In [35]:
"""### Sentiment Analyzer"""

def company_scores(sentence): #returns a dictionary with company names in interest with compound score 
    sentiment_dict = get_sentiment(sentence)
    return sentiment_dict


def apply_vadersentiment(df):
    df['modified']=df['new_text'].apply(lambda x: company_scores(x))
    #df['sentiment_class']=df['sentiment'].apply(lambda x: 'positive' if x['compound']>0.5 else ('negative' if x['compound']<-0.5 else 'neutral'))
    #df['compound score']=df['sentiment'].apply(lambda x: x['compound'])
    return df

Obtain all stocks that we are interested in:

In [33]:
apply_vadersentiment(full_comments_df)

,Date,text,new_text,favorited,retweeted,lang,quoted_status,Stock_Name,sentiment,sentiment_class,compound score,Stock Price,remove,modified
1,10/24/2021,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦","RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!",FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.6825}",positive,0.68,169.8,-1,-1
2,10/24/2021,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,We are going through the beginning stages of everything Ive said in the last year about the economy. Im no genius,FALSE,FALSE,en,0,GME,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}",negative,-0.3,169.8,-1,-1
4,10/24/2021,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,The largest mining company in the world\nGet Bonus 40% \n\n\n$elon $doge $btc $tsla $amc $gme,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.5423}",positive,0,169.8,-1,-1
5,10/24/2021,Crush them! Good luck to $AMC $GME we will soon unite $DWAC https://t.co/cA8Z1sYXtT,Crush them! Good luck to $AMC $GME we will soon unite $DWAC,FALSE,FALSE,en,"{'created_at': 'Sun Oct 24 17:07:17 +0000 2021', 'id': 1452320805132677125, 'id_str': '1452320805132677125', 'text': 'CITADEL GOES TO COURT TOMORROW TO SAY THEY ARE PROTECTING RETAIL INVESTORS.\n\nKEEP IN MIND THE SEC ALREADY HAS A SYSâ€¦ https://t.co/WQkCUvsLnX', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/WQkCUvsLnX', 'expanded_url': 'https://twitter.com/i/web/status/1452320805132677125', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389833864122613761, 'id_str': '1389833864122613761', 'name': 'AMC 2 Moon', 'screen_name': 'AMC_Apee', 'location': 'MoooooooNðŸš€', 'description': 'FOLLOW ME IF YOU WANNA BE RICH! ðŸ’°ðŸš€', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 18218, 'friends_count': 1932, 'listed_count': 58, 'created_at': 'Wed May 05 06:46:46 +0000 2021', 'favourites_count': 23749, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 12259, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1389833864122613761/1634276127', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': False, 'following': None, 'follow_request_sent': None, 'notifications': None, 'translator_type': 'none', 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 112, 'favorite_count': 500, 'favorited': False, 'retwee

In [29]:
full_comments_df

,Date,text,new_text,favorited,retweeted,lang,quoted_status,Stock_Name,sentiment,sentiment_class,compound score,Stock Price,remove,modified
1,10/24/2021,"RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!â€¦","RT @soFloTrading: SUNDAY GIVEAWAY TIME!\n\nOctober 18, 2021 - Weekly Trade Performance\n\nToday is the last day to lock in a FREE weekly trial!",FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.6825}",positive,0.68,169.8,-1,-1
2,10/24/2021,We are going through the beginning stages of everything Iâ€™ve said in the last year about the economy. Iâ€™m no geniusâ€¦ https://t.co/DaBpCTtQh6,We are going through the beginning stages of everything Ive said in the last year about the economy. Im no genius,FALSE,FALSE,en,0,GME,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}",negative,-0.3,169.8,-1,-1
4,10/24/2021,The largest mining company in the world\nGet Bonus 40% â¬‡ï¸\nhttps://t.co/kVp59h2td0\n\n$elon $doge $btc $tsla $amc $gmeâ€¦ https://t.co/I4ZfSukZaP,The largest mining company in the world\nGet Bonus 40% \n\n\n$elon $doge $btc $tsla $amc $gme,FALSE,FALSE,en,0,GME,"{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.5423}",positive,0,169.8,-1,-1
5,10/24/2021,Crush them! Good luck to $AMC $GME we will soon unite $DWAC https://t.co/cA8Z1sYXtT,Crush them! Good luck to $AMC $GME we will soon unite $DWAC,FALSE,FALSE,en,"{'created_at': 'Sun Oct 24 17:07:17 +0000 2021', 'id': 1452320805132677125, 'id_str': '1452320805132677125', 'text': 'CITADEL GOES TO COURT TOMORROW TO SAY THEY ARE PROTECTING RETAIL INVESTORS.\n\nKEEP IN MIND THE SEC ALREADY HAS A SYSâ€¦ https://t.co/WQkCUvsLnX', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/WQkCUvsLnX', 'expanded_url': 'https://twitter.com/i/web/status/1452320805132677125', 'display_url': 'twitter.com/i/web/status/1â€¦', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389833864122613761, 'id_str': '1389833864122613761', 'name': 'AMC 2 Moon', 'screen_name': 'AMC_Apee', 'location': 'MoooooooNðŸš€', 'description': 'FOLLOW ME IF YOU WANNA BE RICH! ðŸ’°ðŸš€', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 18218, 'friends_count': 1932, 'listed_count': 58, 'created_at': 'Wed May 05 06:46:46 +0000 2021', 'favourites_count': 23749, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 12259, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1405293247853826049/yE2Ap9HE_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1389833864122613761/1634276127', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': False, 'following': None, 'follow_request_sent': None, 'notifications': None, 'translator_type': 'none', 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 112, 'favorite_count': 500, 'favorited': False, 'retwee